# 문장학습(GSCV를 이용한 SVM)

In [111]:
from konlpy.tag import *
kkma=Kkma()
tw=Twitter()

```
1. 좌회전
2. 우회전
3. 좌걷기
4. 우걷기
5. 앞걷기
6. 뒤걷기
7. 공격
```

In [38]:
motionDict={1:'좌회전',2:'우회전',3:'좌걷기',4:'우걷기',5:'앞걷기',6:'뒤걷기',7:'공격'}

In [156]:
import csv
Command=[]
Answer=[]
f = open('command.csv', 'r', encoding='utf-8')
rdr = csv.reader(f)
next(rdr)
for line in rdr:
    Command.append(line[0])
    Answer.append(int(line[1]))
f.close()    

In [157]:
x_text_list=[]
y_text_list=[]
for nowC,nowA in zip(Command,Answer):
    tmp=''
    arr=tw.pos(nowC)
    arr2=kkma.pos(nowC)
    for pos,pos2 in zip(arr,arr2):
        #print(pos2)
        if pos[1] == 'Noun' or pos[1] == 'Verb':
            tmp=tmp+pos[0]+' '
        if pos2[1][0] =='N' or pos[1][0] == 'V':
            tmp=tmp+pos2[0]+' '
    x_text_list.append(tmp)
    y_text_list.append(nowA)
print(x_text_list, y_text_list)

f = open("x_text_list.out", 'w')
for now in x_text_list:
    f.write(now)
    f.write('\n')
f.close()


['왼쪽 왼쪽 돌아 돋 ', '좌 좌 회전 회전 ', '좌 좌 돌아 돋 ', '왼쪽 왼쪽 회전 회전 ', '우 우로 회전 ', '우 우로 돌아 아 ', '오른쪽 오른쪽 회전 회전 ', '왼쪽 왼쪽 걸어 걷 ', '왼쪽 왼쪽 가 가 ', '왼쪽 왼쪽 전진 전진 ', '좌 좌 전진 전진 ', '오른쪽 오른쪽 걸어 걷 ', '오른쪽 오른쪽 가 가 ', '오른쪽 오른쪽 전진 전진 ', '우 우로 전진 ', '앞 앞 전진 전진 ', '앞 앞 가 가 ', '뒤 뒤 가 가 ', '뒤 뒤 걸어 걷 ', '공격 공격 ', '펀치 펀치 ', '주먹 주먹 때려 때리 ', '공격 공격 ', '때려 때리 '] [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 6, 6, 7, 7, 7, 7, 7]


In [158]:
from sklearn.model_selection import train_test_split
import re
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
import json
import nltk
vectorizer = CountVectorizer()

In [159]:
svc_param = np.logspace(-2,0,1000)
gsvc = GridSearchCV(LinearSVC(), param_grid= {'C': svc_param}, cv = 2, n_jobs = 4)
print(x_text_list)
vectorizer.fit_transform(x_text_list)

['왼쪽 왼쪽 돌아 돋 ', '좌 좌 회전 회전 ', '좌 좌 돌아 돋 ', '왼쪽 왼쪽 회전 회전 ', '우 우로 회전 ', '우 우로 돌아 아 ', '오른쪽 오른쪽 회전 회전 ', '왼쪽 왼쪽 걸어 걷 ', '왼쪽 왼쪽 가 가 ', '왼쪽 왼쪽 전진 전진 ', '좌 좌 전진 전진 ', '오른쪽 오른쪽 걸어 걷 ', '오른쪽 오른쪽 가 가 ', '오른쪽 오른쪽 전진 전진 ', '우 우로 전진 ', '앞 앞 전진 전진 ', '앞 앞 가 가 ', '뒤 뒤 가 가 ', '뒤 뒤 걸어 걷 ', '공격 공격 ', '펀치 펀치 ', '주먹 주먹 때려 때리 ', '공격 공격 ', '때려 때리 ']


<24x12 sparse matrix of type '<class 'numpy.int64'>'
	with 35 stored elements in Compressed Sparse Row format>

In [160]:
gsvc.fit(vectorizer.transform(x_text_list), y_text_list)

GridSearchCV(cv=2, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'C': array([ 0.01   ,  0.01005, ...,  0.9954 ,  1.     ])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [161]:
print(gsvc.best_score_, gsvc.best_params_)

0.5 {'C': 0.01}


In [162]:
joblib.dump(gsvc, 'model.pkl')

['model.pkl']

# 머신러닝 테스트 코드

In [192]:
testW=['앞으로 가자','뒤로 가봐','왼쪽으로 걸어','왼쪽으로 회전','좌 회전','오른쪽으로 걸어', '오른쪽으로 회전','앞 때려','공격해','왼쪽 가즈아','오른쪽 가즈아','심규 때리고 싶다','빨리 앞으로 가라고','앞으로가라','왼쪽으로 돌지말고 오른쪽으로 가자','오른쪽으로 가지말고 왼쪽으로 돌아라','꽃길만 걷자'] 

In [193]:
test=[]
test2=[]
for nowT in testW:
    tmp=''
    arr=tw.pos(nowT)
    arr2=kkma.pos(nowT)
    for pos,pos2 in zip(arr,arr2):
        #print(pos)
        if pos[1] == 'Noun' or pos[1] == 'Verb':
            tmp=tmp+pos[0]+' '
        if pos2[1][0] =='N' or pos2[1][0] == 'V':
            tmp=tmp+pos2[0]+' '
    test.append(tmp)
print(test)

['앞 앞 가자 가자 ', '뒤 뒤 가봐 가보 ', '왼쪽 왼쪽 걸어 걷 ', '왼쪽 왼쪽 회전 회전 ', '좌 좌 회전 회전 ', '오른쪽 오른쪽 걸어 걷 ', '오른쪽 오른쪽 회전 회전 ', '앞 앞 때려 때리 ', '공격 공격 ', '왼쪽 왼쪽 가즈 가 ', '오른쪽 오른쪽 가즈 가 ', '심규 심규 때리 때리 싶다 싶 ', '앞 앞 가라 가라 ', '앞 앞 가라 가라 ', '왼쪽 왼쪽 돌지 돌 오른쪽 말 가자 오른쪽 ', '오른쪽 오른쪽 가지 가 왼쪽 말 돌아 왼쪽 ', '꽃길 꽃 길 걷자 ']


In [194]:
vectorizer.transform(test)

<17x12 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [195]:
pred_list = gsvc.predict(vectorizer.transform(test))

In [196]:
print(pred_list)

[7 7 3 1 1 4 4 7 7 3 4 7 7 7 4 4 7]


In [197]:
for now,now2 in zip(pred_list,testW):
    print("예측 : ",motionDict[now],"\t 정답 : ",now2)

예측 :  공격 	 정답 :  앞으로 가자
예측 :  공격 	 정답 :  뒤로 가봐
예측 :  좌걷기 	 정답 :  왼쪽으로 걸어
예측 :  좌회전 	 정답 :  왼쪽으로 회전
예측 :  좌회전 	 정답 :  좌 회전
예측 :  우걷기 	 정답 :  오른쪽으로 걸어
예측 :  우걷기 	 정답 :  오른쪽으로 회전
예측 :  공격 	 정답 :  앞 때려
예측 :  공격 	 정답 :  공격해
예측 :  좌걷기 	 정답 :  왼쪽 가즈아
예측 :  우걷기 	 정답 :  오른쪽 가즈아
예측 :  공격 	 정답 :  심규 때리고 싶다
예측 :  공격 	 정답 :  빨리 앞으로 가라고
예측 :  공격 	 정답 :  앞으로가라
예측 :  우걷기 	 정답 :  왼쪽으로 돌지말고 오른쪽으로 가자
예측 :  우걷기 	 정답 :  오른쪽으로 가지말고 왼쪽으로 돌아라
예측 :  공격 	 정답 :  꽃길만 걷자
